In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="0"

In [2]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
gpus[0]
tf.config.experimental.set_memory_growth(gpus[0], True)

2022-03-10 12:29:30.713802: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-10 12:29:30.720932: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-10 12:29:30.721539: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
import os
import platform
from absl import app
from absl import flags
from absl import logging
#import tensorflow as tf

import dataloader
import hparams_config
import utils
from tf2 import train_lib
from tf2 import util_keras

In [4]:
import json

#train_json = json.load("_logs/json_.json")
#val_json = json.load("_logs/val_json_.json")

In [5]:
#flags.FLAGS._parse_flags(args=args)
model_name = 'efficientdet-d2'

In [6]:
config = hparams_config.get_detection_config(model_name)

config.num_epochs = 300
config.batch_size = 4
#config.batch = 2
config.moving_average_decay = None
config.image_size = (768,768)
config.model_dir = '/host/04 - Plovila Detection/data/01 active - open images data and labels'
config.log_dir = '_logs'
config.val_json_file = None
config.steps_per_execution = 1
config.max_instances_per_image = 200
config.strategy = 'gpus'

## TODO: AUTOMATIC NUM SAMPLES CALC
config.steps_per_epoch = 21648//config.batch_size
config.num_classes = 1
validation_steps = 3821//config.batch_size

In [7]:
#train_file_pattern="_logs/tfrecords/train-*.tfrecord"
#val_file_pattern="_logs/tfrecords/val-*.tfrecord"

In [8]:
use_fake_data = False
debug = False

ds_strategy = tf.distribute.OneDeviceStrategy('device:GPU:0')


2022-03-10 12:29:31.396664: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-10 12:29:31.397129: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-10 12:29:31.397752: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-10 12:29:31.398205: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [9]:
def get_dataset(is_training, config):
    file_pattern = (
        "_logs/tfrecords/train-*.tfrecord"
        if is_training else "_logs/tfrecords/val-*.tfrecord")
    if not file_pattern:
      raise ValueError('No matching files.')

    return dataloader.InputReader(
        file_pattern,
        is_training=is_training,
        use_fake_data=use_fake_data,
        max_instances_per_image=config.max_instances_per_image,
        debug=debug)(config.as_dict())

In [10]:
train_dataset = get_dataset(True, config)

In [11]:
train_dataset

<RepeatDataset shapes: ((4, 768, 768, 3), {mean_num_positives: (4, 1), cls_targets_3: (4, 96, 96, 9), box_targets_3: (4, 96, 96, 36), cls_targets_4: (4, 48, 48, 9), box_targets_4: (4, 48, 48, 36), cls_targets_5: (4, 24, 24, 9), box_targets_5: (4, 24, 24, 36), cls_targets_6: (4, 12, 12, 9), box_targets_6: (4, 12, 12, 36), cls_targets_7: (4, 6, 6, 9), box_targets_7: (4, 6, 6, 36), source_ids: (4,), groundtruth_data: (4, 200, 7), image_scales: (4,), image_masks: (4, 0)}), types: (tf.float32, {mean_num_positives: tf.float32, cls_targets_3: tf.int32, box_targets_3: tf.float32, cls_targets_4: tf.int32, box_targets_4: tf.float32, cls_targets_5: tf.int32, box_targets_5: tf.float32, cls_targets_6: tf.int32, box_targets_6: tf.float32, cls_targets_7: tf.int32, box_targets_7: tf.float32, source_ids: tf.float32, groundtruth_data: tf.float32, image_scales: tf.float32, image_masks: tf.float32})>

In [12]:
val_dataset = get_dataset(False, config) 

In [13]:
val_dataset

<PrefetchDataset shapes: ((4, 768, 768, 3), {mean_num_positives: (4, 1), cls_targets_3: (4, 96, 96, 9), box_targets_3: (4, 96, 96, 36), cls_targets_4: (4, 48, 48, 9), box_targets_4: (4, 48, 48, 36), cls_targets_5: (4, 24, 24, 9), box_targets_5: (4, 24, 24, 36), cls_targets_6: (4, 12, 12, 9), box_targets_6: (4, 12, 12, 36), cls_targets_7: (4, 6, 6, 9), box_targets_7: (4, 6, 6, 36), source_ids: (4,), groundtruth_data: (4, 200, 7), image_scales: (4,), image_masks: (4, 0)}), types: (tf.float32, {mean_num_positives: tf.float32, cls_targets_3: tf.int32, box_targets_3: tf.float32, cls_targets_4: tf.int32, box_targets_4: tf.float32, cls_targets_5: tf.int32, box_targets_5: tf.float32, cls_targets_6: tf.int32, box_targets_6: tf.float32, cls_targets_7: tf.int32, box_targets_7: tf.float32, source_ids: tf.float32, groundtruth_data: tf.float32, image_scales: tf.float32, image_masks: tf.float32})>

In [14]:
def init_experimental(config):
  """Serialize train config to model directory."""
  tf.io.gfile.makedirs(config.log_dir)
  config_file = os.path.join(config.model_dir, 'config.yaml')
  if not tf.io.gfile.exists(config_file):
    tf.io.gfile.GFile(config_file, 'w').write(str(config))

In [15]:
def setup_model(model, config):
  """Build and compile model."""
  model.build((None, *config.image_size, 3))
  model.compile(
      steps_per_execution=config.steps_per_execution,
      optimizer=train_lib.get_optimizer(config.as_dict()),
      loss={
          train_lib.BoxLoss.__name__:
              train_lib.BoxLoss(
                  config.delta, reduction=tf.keras.losses.Reduction.NONE),
          train_lib.BoxIouLoss.__name__:
              train_lib.BoxIouLoss(
                  config.iou_loss_type,
                  config.min_level,
                  config.max_level,
                  config.num_scales,
                  config.aspect_ratios,
                  config.anchor_scale,
                  config.image_size,
                  reduction=tf.keras.losses.Reduction.NONE),
          train_lib.FocalLoss.__name__:
              train_lib.FocalLoss(
                  config.alpha,
                  config.gamma,
                  label_smoothing=config.label_smoothing,
                  reduction=tf.keras.losses.Reduction.NONE),
          tf.keras.losses.SparseCategoricalCrossentropy.__name__:
              tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
      })
  return model

In [ ]:
# set mixed precision policy by keras api.
precision = utils.get_precision(config.strategy, config.mixed_precision)
policy = tf.keras.mixed_precision.Policy(precision)
tf.keras.mixed_precision.set_global_policy(policy)

In [ ]:
with ds_strategy.scope():
  
  #Load and compile model
  model = train_lib.EfficientDetNetTrain(config=config)
  model.layers[0].trainable = False
  model = setup_model(model, config)
  
  #Load checkpoints if they exist
  if tf.train.latest_checkpoint(config.log_dir):
    ckpt_path = tf.train.latest_checkpoint(config.log_dir)
    util_keras.restore_ckpt(model,
                            ckpt_path,
                            config.moving_average_decay)
  init_experimental(config)
  
  #Train model
  model.fit(train_dataset,
          epochs=config.num_epochs,
          steps_per_epoch=config.steps_per_epoch,
          initial_epoch=model.optimizer.iterations.numpy() // config.steps_per_epoch,
          callbacks=train_lib.get_callbacks(config.as_dict(), val_dataset),
          validation_data=val_dataset,
          validation_steps=validation_steps)

In [ ]:
dir(val_dataset)

In [16]:
import matplotlib.pyplot as plt
from visualize.vis_utils import draw_bounding_boxes_on_image_tensors

In [17]:
import numpy as np
import cv2

In [35]:
cls = {1: {1: 'Boat','name': 'Boat'}}


In [37]:
for images,labels in val_dataset.take(1):
  
    print(images.shape)
    print(labels['groundtruth_data'][:].shape)
    boxes = np.uint8(labels['groundtruth_data'][:,:,:4].numpy())
    confidence = labels['groundtruth_data'][:,:,5].numpy()
    
    classes = labels['groundtruth_data'][:,:,6].numpy()
    print(images.shape,boxes.shape,confidence.shape,classes.shape)

    break

<dtype: 'float32'>
(4, 200, 7)
(4, 768, 768, 3) (4, 200, 4) (4, 200) (4, 200)


In [ ]:
print(labels.keys())

In [ ]:
print(labels['image_scales'])

In [ ]:
#dir(model.layers[3].cells[1])

In [ ]:
#dir(model.layers[3].cells[1].fnodes[0])

In [ ]:
config.as_dict().get('map_freq', None)

In [ ]:
config.as_dict().get('moving_average_decay', None)

In [ ]:
model.summary()

In [ ]:
if None:
  print('g')
else:
  print('b')